In [1]:
pwd

'C:\\Users\\Baron\\Desktop\\EE_297_Repo\\EE_297\\ML_PATH_EE297\\EE297_env\\01_main\\03_lanes_code'

In [2]:
pwd

'C:\\Users\\Baron\\Desktop\\EE_297_Repo\\EE_297\\ML_PATH_EE297\\EE297_env\\01_main\\03_lanes_code'

In [3]:
pwd

'C:\\Users\\Baron\\Desktop\\EE_297_Repo\\EE_297\\ML_PATH_EE297\\EE297_env\\01_main\\03_lanes_code'

### IMPORTS

In [4]:
#1. Import Necessary Liobraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import random
import shutil
from tqdm.auto import tqdm as notebook_tqdm
import tqdm
import tensorflow as tf

import pandas as pd
import numpy as np
from PIL import Image

import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


from keras.models import Sequential
from keras.layers import Input, AveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1

from keras.optimizers import Adam
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#import torchvision.transforms as transforms

#import warnings
#warnings.filterwarnings("ignore")


### RESIZING IMAGE (SW + HW)

In [5]:
from PIL import Image

# Input and output paths
input_path = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og.jpg"
output_path = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og_224.jpg"

# Open the image
image = Image.open(input_path)

# Resize to 224x224
resized_image = image.resize((224, 224))

# Save the resized image
resized_image.save(output_path)

print(f"Image saved to {output_path}")

Image saved to C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og_224.jpg


### CONVERTS IMAGE TO .TXT FILE (HW) 

In [6]:
import cv2
import numpy as np
import os

# ==== USER INPUT ====
input_path = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_og_224.jpg"
output_basename = "img_rgb_224.txt"  # Output .txt filename
target_size = (224, 224)             # Resize target

# ==== LOAD IMAGE ====
img = cv2.imread(input_path)
if img is None:
    raise FileNotFoundError(f"Could not load image at: {input_path}")

img = cv2.resize(img, target_size)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR → RGB
img = img.astype(np.float32) / 255.0        # Normalize to [0,1]

# ==== NORMALIZE WITH MEAN & STD (ImageNet-style) ====
mean = np.array([0.485, 0.456, 0.406])  # R, G, B
std  = np.array([0.229, 0.224, 0.225])

img = (img - mean[None, None, :]) / std[None, None, :]  # broadcast across H, W

# ==== WRITE TO TXT ====
output_dir = os.path.dirname(input_path)
output_path = os.path.join(output_dir, output_basename)

with open(output_path, "w") as f:
    for c in range(3):  # R, G, B
        for i in range(target_size[1]):
            for j in range(target_size[0]):
                f.write(f"{img[i, j, c]:.6f} ")
            f.write("\n")

print(f"[✓] Normalized image written to .txt for hardware input:\n{output_path}")

[✓] Normalized image written to .txt for hardware input:
C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_rgb_224.txt


### MODEL CREATION (SW)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class LaneSegOnlyMobileNetV2(nn.Module):
    def __init__(self, pretrained=True, freeze_stem=False):
        super().__init__()
        # ---- Encoder: MobileNetV2 ----
        weights = models.MobileNet_V2_Weights.IMAGENET1K_V1 if pretrained else None
        m = models.mobilenet_v2(weights=weights)

        # Feature extractor (ends at last conv-bn-relu block)
        self.encoder = m.features              # output: (B, 1280, 7, 7) for 224x224
        enc_out_ch = 1280

        if freeze_stem:
            # freeze early, cheap speed/regularization trick
            for p in list(self.encoder.parameters())[:]:
                p.requires_grad = False

        # ---- Lightweight decoder (no skip connections) ----
        self.seg_head = nn.Sequential(
            nn.ConvTranspose2d(enc_out_ch, 256, kernel_size=2, stride=2),  # 7 -> 14
            nn.BatchNorm2d(256), nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),         # 14 -> 28
            nn.BatchNorm2d(128), nn.ReLU(inplace=True),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),          # 28 -> 56
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),           # 56 -> 112
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),

            nn.Conv2d(32, 1, kernel_size=1),  # logits (B,1,112,112)
        )

    def forward(self, x):
        H, W = x.shape[-2:]
        feats = self.encoder(x)                 # (B,1280,7,7) at 224x224
        seg   = self.seg_head(feats)           # (B,1,112,112)
        seg   = F.interpolate(seg, size=(H, W), mode="bilinear", align_corners=False)
        return seg


### LOAD IN TRAINED WEIGHTS FILE (SW)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LaneSegOnlyMobileNetV2()
model.load_state_dict(torch.load("lane_seg_weights.pth"))
model.to(device)
model.eval()  # set to inference mode

C:\Users\Baron\AppData\Local\Temp\ipykernel_5132\2694822331.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_seg_weights.pth"))


LaneSegOnlyMobileNetV2(
  (encoder): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2

In [9]:
for name, param in model.named_parameters():
    print(name, param.shape)

encoder.0.0.weight torch.Size([32, 3, 3, 3])
encoder.0.1.weight torch.Size([32])
encoder.0.1.bias torch.Size([32])
encoder.1.conv.0.0.weight torch.Size([32, 1, 3, 3])
encoder.1.conv.0.1.weight torch.Size([32])
encoder.1.conv.0.1.bias torch.Size([32])
encoder.1.conv.1.weight torch.Size([16, 32, 1, 1])
encoder.1.conv.2.weight torch.Size([16])
encoder.1.conv.2.bias torch.Size([16])
encoder.2.conv.0.0.weight torch.Size([96, 16, 1, 1])
encoder.2.conv.0.1.weight torch.Size([96])
encoder.2.conv.0.1.bias torch.Size([96])
encoder.2.conv.1.0.weight torch.Size([96, 1, 3, 3])
encoder.2.conv.1.1.weight torch.Size([96])
encoder.2.conv.1.1.bias torch.Size([96])
encoder.2.conv.2.weight torch.Size([24, 96, 1, 1])
encoder.2.conv.3.weight torch.Size([24])
encoder.2.conv.3.bias torch.Size([24])
encoder.3.conv.0.0.weight torch.Size([144, 24, 1, 1])
encoder.3.conv.0.1.weight torch.Size([144])
encoder.3.conv.0.1.bias torch.Size([144])
encoder.3.conv.1.0.weight torch.Size([144, 1, 3, 3])
encoder.3.conv.1.1.we

### (encoder): Sequential( HW Conv -> FOLD (BatchNorm) -> Relu)
1. Save Weights File (HW)

In [10]:
# 1) Get the encoder stage shown in your screenshot

#    It's a Conv2dNormActivation with (0)=Conv2d, (1)=BatchNorm2d, (2)=ReLU6
# encoder[0] = Conv2dNormActivation: [0]=Conv2d(bias=False), [1]=BatchNorm2d, [2]=ReLU6
enc0 = model.encoder[0]
conv: nn.Conv2d = enc0[0]
bn:   nn.BatchNorm2d = enc0[1]

assert isinstance(conv, nn.Conv2d)
assert isinstance(bn,   nn.BatchNorm2d)

print("Conv shape (O,I,K,K):", tuple(conv.weight.shape))
print("BN shapes:", tuple(bn.weight.shape), tuple(bn.bias.shape))

# Setting up BatchNorm Math Fold:
# Get tensors (PyTorch conv layout: [O,I,K,K])
W = conv.weight.detach().cpu().numpy().astype("float32")          # (O,I,K,K)
gamma = bn.weight.detach().cpu().numpy().astype("float32")        # (O,)
beta  = bn.bias.detach().cpu().numpy().astype("float32")          # (O,)
mean  = bn.running_mean.detach().cpu().numpy().astype("float32")  # (O,)
var   = bn.running_var.detach().cpu().numpy().astype("float32")   # (O,)
eps   = float(bn.eps)

# Per-output scale
s = (gamma / np.sqrt(var + eps)).astype("float32")  # (O,)

# Fold into conv (bias=False case)
W_fold = (W * s[:, None, None, None]).astype("float32")  # broadcast scale over I,K,K
b_fold = (beta - mean * s).astype("float32")             # (O,)

# If your HLS expects (K,K,IN_C,OUT_C) layout:
W_hls = np.transpose(W_fold, (2, 3, 1, 0))  # (K,K,I,O)

Conv shape (O,I,K,K): (32, 3, 3, 3)
BN shapes: (32,) (32,)


In [11]:
import os
import numpy as np

OUT_DIR = "weights"      # relative path in your project
CTYPE   = "data_t"       # ap_fixed typedef in your HLS

HEADER_INCLUDE = r'#include "C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\lane_seg_top.h"'

def write_header_plain_numbers(array: np.ndarray, var_name: str, ctype: str, out_path: str):
    """
    Writes a header with:
      - #pragma once
      - your include path
      - declaration: data_t var[... ] = { ... };
      - numbers printed as plain literals (no 'f', no '(data_t)' casts)
    """
    array = np.asarray(array, dtype=np.float32)
    dims  = "".join(f"[{d}]" for d in array.shape)

    def fmt(a, indent=0):
        if a.ndim == 1:
            # no 'f' and no casts; compact scientific where needed
            return "{" + ",".join(f"{x:.8f}" for x in a.tolist()) + "}"
        body = ",\n".join((" "*(indent+2)) + fmt(x, indent+2) for x in a)
        return "{\n" + body + "\n" + (" "*indent) + "}"

    with open(out_path, "w") as f:
        f.write("#pragma once\n\n")
        f.write(f'{HEADER_INCLUDE}\n\n')
        f.write("// Auto-generated from encoder[0] with BN folded\n\n")
        # no 'const' here
        f.write(f"{ctype} {var_name}{dims} = ")
        f.write(fmt(array))
        f.write(";\n")

    # print full path (one-liner)
    print(os.path.abspath(out_path))

# ---- call these with your already-computed arrays ----
# W_hls: (K,K,IN_C,OUT_C), b_fold: (OUT_C,)
os.makedirs(OUT_DIR, exist_ok=True)
write_header_plain_numbers(W_hls, "conv0_w", CTYPE, os.path.join(OUT_DIR, "encoder_conv0_w.h"))
write_header_plain_numbers(b_fold, "conv0_b", CTYPE, os.path.join(OUT_DIR, "encoder_conv0_b.h"))


C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\encoder_conv0_w.h
C:\Users\Baron\Desktop\EE_297_Repo\EE_297\ML_PATH_EE297\EE297_env\01_main\03_lanes_code\weights\encoder_conv0_b.h


In [12]:
### WEIGHT COMPARISON

### SW output of Conv Stage 0 Comparison)

In [26]:
import torch
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
from PIL import Image
import os

# -------------------------------
# CONFIGURATION
# -------------------------------
INT_BITS = 4
FRAC_BITS = 12
IMG_PATH = "C:/Users/Baron/Desktop/EE_297_Repo/EE_297/hardware_imp/vitis_hls/lane_seg_hls/test_imgs/img_rgb_224.txt"
WEIGHT_SAVE_DIR = "software_output"
os.makedirs(WEIGHT_SAVE_DIR, exist_ok=True)

# -------------------------------
# FIXED-POINT QUANTIZATION
# -------------------------------
def quantize_fixed(x, int_bits=INT_BITS, frac_bits=FRAC_BITS):
    scale = 2 ** frac_bits
    max_val = (2 ** (int_bits - 1)) - 1 / scale
    min_val = -(2 ** (int_bits - 1))
    return np.clip(np.round(x * scale) / scale, min_val, max_val)

# -------------------------------
# LOAD IMAGE (.txt format, channel-major: R→G→B blocks)
# -------------------------------
def load_txt_tensor(filepath, H=224, W=224, C=3):
    arr = np.loadtxt(filepath, dtype=np.float32)
    if arr.size != C * H * W:
        raise ValueError(f"Unexpected image size: got {arr.size} elements")

    arr = arr.reshape(C, H, W)       # (C,H,W), channel-major
    arr = np.expand_dims(arr, axis=0)  # (1,C,H,W) for torch
    return torch.tensor(arr, dtype=torch.float32)
    
# -------------------------------
# MAIN PIPELINE
# -------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneSegOnlyMobileNetV2(pretrained=False)
model.load_state_dict(torch.load("lane_seg_weights.pth", map_location=device))
model.to(device)
model.eval()

# Extract encoder[0] weights and BN params
enc0 = model.encoder[0]
conv: torch.nn.Conv2d = enc0[0]
bn:   torch.nn.BatchNorm2d = enc0[1]

# Fold BN into conv weights
W = conv.weight.detach().cpu().numpy().astype(np.float32)         # (O,I,K,K)
gamma = bn.weight.detach().cpu().numpy().astype(np.float32)
beta  = bn.bias.detach().cpu().numpy().astype(np.float32)
mean  = bn.running_mean.detach().cpu().numpy().astype(np.float32)
var   = bn.running_var.detach().cpu().numpy().astype(np.float32)
eps   = float(bn.eps)

s = gamma / np.sqrt(var + eps)
W_fold = W * s[:, None, None, None]
b_fold = beta - mean * s

# Quantize weights and biases
W_fold_q = quantize_fixed(W_fold)
b_fold_q = quantize_fixed(b_fold)
W_torch = torch.tensor(W_fold_q).to(device)
b_torch = torch.tensor(b_fold_q).to(device)

# Load image directly from .txt
x = load_txt_tensor(IMG_PATH).to(device)

# Quantize input
x_q = quantize_fixed(x.cpu().numpy())
x_q = torch.tensor(x_q).to(device)

# Run convolution + ReLU6
with torch.no_grad():
    y = F.conv2d(x_q, W_torch, bias=b_torch, stride=2, padding=1)
    y = torch.clamp(y, 0, 6)

# Save results
y_np = y.squeeze(0).cpu().numpy()
np.save(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder0_output.npy"), y_np)

with open(os.path.join(WEIGHT_SAVE_DIR, "sw_encoder0_output.txt"), "w") as f:
    for c in range(32):
        for i in range(112):
            for j in range(112):
                f.write(f"{y_np[c][i][j]:.6f} ")
            f.write("\n")
        f.write("\n")

print("✅ Software output with fixed-point weights, input, and arithmetic saved.")


C:\Users\Baron\AppData\Local\Temp\ipykernel_5132\2702204533.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lane_seg_weights.pth", map

✅ Software output with fixed-point weights, input, and arithmetic saved.


In [27]:
import torch.nn.functional as F

# -----------------------------------------
# Step 1: Manually apply zero-padding to input
# Same as: padded[y + PAD][x + PAD][c] = input[y][x][c];
# -----------------------------------------
PAD = 1  # Same as HLS
x_padded = F.pad(x_q, pad=(PAD, PAD, PAD, PAD), mode='constant', value=0)  # shape: (1, 3, 226, 226)

# -----------------------------------------
# Step 2: Debug the very first conv output: [oh=0, ow=0, oc=0]
# -----------------------------------------
STRIDE = 2
K = 3
iy = 0 * STRIDE
ix = 0 * STRIDE
oc = 0

bias_debug = b_torch[oc].item()
print(f"DEBUG: bias[0] = {bias_debug:.6f}")

w_debug = W_torch[oc, 0, 0, 0].item()
print(f"DEBUG: weights[0][0][0][0] = {w_debug:.6f}")

sum_debug = bias_debug
for ky in range(K):
    for kx in range(K):
        for ic in range(3):  # IN_C = 3
            a = x_padded[0, ic, iy + ky, ix + kx].item()
            b = W_torch[oc, ic, ky, kx].item()
            p = a * b
            sum_debug += p
            print(f"DEBUG: a={a:.6f} * b={b:.6f} => {p:.6f} | sum={sum_debug:.6f}")

# Final ReLU6
sum_clamped = max(0, min(6, sum_debug))
print(f"DEBUG: output[0][0][0] = {sum_clamped:.6f}")


DEBUG: bias[0] = -0.022705
DEBUG: weights[0][0][0][0] = 0.006592
DEBUG: a=0.000000 * b=0.006592 => 0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=0.003662 => 0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=-0.016113 => -0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=-0.003418 => -0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=-0.004395 => -0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=-0.012939 => -0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=0.007568 => 0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=0.007812 => 0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=-0.006836 => -0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=0.018555 => 0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=0.010986 => 0.000000 | sum=-0.022705
DEBUG: a=0.000000 * b=-0.007568 => -0.000000 | sum=-0.022705
DEBUG: a=-1.501465 * b=-0.015869 => 0.023827 | sum=0.001122
DEBUG: a=-1.265381 * b=-0.020264 => 0.025641 | sum=0.026763
DEBUG: a=-1.002686 * b=-0.018311 => 0.018360 | sum=0.045123
DEBUG: a=-1.518555 * b=0.003174 => 

In [15]:
### Datatype Conversion Understand:
import numpy as np

# ==== CONFIG ====
input_txt = r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\img_rgb_224.txt"

TOTAL_BITS = 16
INT_BITS = 4
FRAC_BITS = TOTAL_BITS - INT_BITS   # = 12
SCALE = 2 ** FRAC_BITS
MIN_VAL = - (2 ** (INT_BITS - 1))          # -8.0
MAX_VAL = (2 ** (INT_BITS - 1)) - (1 / SCALE)  # 7.999755859375

def float_to_ap_fixed(x):
    """Simulate ap_fixed<16,4>: quantize float to fixed-point representation."""
    # Saturate to representable range
    x_clamped = np.clip(x, MIN_VAL, MAX_VAL)
    # Scale to integer
    fixed_val = np.round(x_clamped * SCALE).astype(np.int32)
    return fixed_val

def ap_fixed_to_float(fixed_val):
    """Convert back to float from fixed-point representation."""
    return fixed_val.astype(np.float32) / SCALE

# ==== LOAD TXT ====
with open(input_txt, "r") as f:
    values = [float(v) for v in f.read().split()]

values = np.array(values, dtype=np.float32)

# ==== CONVERT TO FIXED & BACK ====
fixed_vals = float_to_ap_fixed(values)
roundtrip_vals = ap_fixed_to_float(fixed_vals)

# ==== INSPECT ====
for i in range(5):
    print(f"Original: {values[i]:.6f}, Fixed: {fixed_vals[i]}, Back-to-float: {roundtrip_vals[i]:.6f}")

# ==== OPTIONAL: error analysis ====
abs_err = np.abs(values - roundtrip_vals)
print(f"\nMax abs error: {abs_err.max():.6f}")
print(f"Mean abs error: {abs_err.mean():.6f}")


Original: -1.501413, Fixed: -6150, Back-to-float: -1.501465
Original: -1.518538, Fixed: -6220, Back-to-float: -1.518555
Original: -1.518538, Fixed: -6220, Back-to-float: -1.518555
Original: -1.535662, Fixed: -6290, Back-to-float: -1.535645
Original: -1.552787, Fixed: -6360, Back-to-float: -1.552734

Max abs error: 0.000122
Mean abs error: 0.000062


In [19]:
import numpy as np

H, W, C = 224, 224, 3
K, STRIDE, PAD = 3, 1, 1

# Load file values into array
with open(r"C:\Users\Baron\Desktop\EE_297_Repo\EE_297\hardware_imp\vitis_hls\lane_seg_hls\test_imgs\\img_rgb_224.txt", "r") as f:
    values = np.array([float(x) for x in f.read().split()], dtype=np.float32)

# Reconstruct channel-major array
img = np.zeros((H, W, C), dtype=np.float32)
idx = 0
for c in range(C):
    for i in range(H):
        for j in range(W):
            img[i, j, c] = values[idx]
            idx += 1

# Map file index for each conv access
def file_index(i, j, c):
    return c * H * W + i * W + j

# Example: first output pixel (oh=0, ow=0)
iy, ix = 0, 0
for ky in range(K):
    for kx in range(K):
        for ic in range(C):
            val = img[iy+ky, ix+kx, ic]
            idx = file_index(iy+ky, ix+kx, ic)
            print(f"a={val:.6f} comes from file index {idx}")


a=-1.501413 comes from file index 0
a=-1.265406 comes from file index 50176
a=-1.002702 comes from file index 100352
a=-1.518538 comes from file index 1
a=-1.282913 comes from file index 50177
a=-1.020131 comes from file index 100353
a=-1.518538 comes from file index 2
a=-1.282913 comes from file index 50178
a=-1.020131 comes from file index 100354
a=-1.689785 comes from file index 224
a=-1.457983 comes from file index 50400
a=-1.194423 comes from file index 100576
a=-1.672660 comes from file index 225
a=-1.440476 comes from file index 50401
a=-1.176993 comes from file index 100577
a=-1.655536 comes from file index 226
a=-1.422969 comes from file index 50402
a=-1.159564 comes from file index 100578
a=-1.775409 comes from file index 448
a=-1.545518 comes from file index 50624
a=-1.281569 comes from file index 100800
a=-1.758284 comes from file index 449
a=-1.528011 comes from file index 50625
a=-1.264139 comes from file index 100801
a=-1.706910 comes from file index 450
a=-1.475490 come